In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

/Users/ale/anaconda3/envs/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/ale/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import_data = pd.read_csv("datasets/imports/imports.csv", skipinitialspace=True, skiprows=1, na_values="?")

In [3]:
import_data.sample(10)

,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.60,...,130,mpfi,3.47,2.68,9.00,111,5000,21,27,16500
159,0,91.0,toyota,gas,std,four,hatchback,fwd,front,95.7,...,98,2bbl,3.19,3.03,9.0,70.0,4800.0,28,34,8358.0
168,2,134.0,toyota,gas,std,two,hardtop,rwd,front,98.4,...,146,mpfi,3.62,3.50,9.3,116.0,4800.0,24,30,11199.0
10,0,188.0,bmw,gas,std,two,sedan,rwd,front,101.2,...,164,mpfi,3.31,3.19,9.0,121.0,4250.0,21,28,20970.0
44,2,NaN,isuzu,gas,std,two,hatchback,rwd,front,96.0,...,119,spfi,3.43,3.23,9.2,90.0,5000.0,24,29,11048.0
83,1,125.0,mitsubishi,gas,std,four,sedan,fwd,front,96.3,...,122,2bbl,3.35,3.46,8.5,88.0,5000.0,25,32,6989.0
33,0,110.0,honda,gas,std,four,sedan,fwd,front,96.5,...,92,1bbl,2.91,3.41,9.2,76.0,6000.0,30,34,7295.0
17,1,98.0,chevrolet,gas,std,two,hatchback,fwd,front,94.5,...,90,2bbl,3.03,3.11,9.6,70.0,5400.0,38,43,6295.0
149,1,87.0,toyota,gas,std,two,hatchback,fwd,front,95.7,...,92,2bbl,3.05,3.03,9.0,62.0,4800.0,31,38,6338.0
166,2,134.0,toyota,gas,std,two,hardtop,rwd,front,98.4,...,146,mpfi,3.62,3.50,9.3,116.0,4800.0,24,30,9639.0
105,0,161.0,peugot,gas,std,four,sedan,rwd,front,107.9,...,120,mpfi,3.46,3.19,8.4,97.0,5000.0,19,24,11900.0


In [4]:
# set missing column names
import_data.columns = [
    "symboling",
    "normalized-losses",
    "make",
    "fuel-type",
    "aspiration",
    "num-of-doors",
    "body-style",
    "drive-wheels",
    "engine-location",
    "wheel-base",
    "length", 
    "width", 
    "height", 
    "curb-weight", 
    "engine-type",
    "num-of-cylinders",
    "engine-size", 
    "fuel-system",
    "bore", 
    "stroke", 
    "compression-ratio", 
    "horsepower",
    "peak-rpm",
    "city-mpg",
    "highway-mpg",
    "price"
]

In [5]:
# split train data into validation data
train_data = import_data.sample(frac=0.8, replace=False)
validation_data = import_data[len(train_data):]

# cleanup data
train_data = train_data[np.isfinite(train_data['price'])]

In [6]:
train_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
102,3,194.0,nissan,gas,std,two,hatchback,rwd,front,91.3,...,181,mpfi,3.43,3.27,9.0,160.0,5200.0,19,25,17199.0
120,1,154.0,plymouth,gas,std,four,sedan,fwd,front,93.7,...,98,2bbl,2.97,3.23,9.4,68.0,5500.0,31,38,7609.0
110,0,161.0,peugot,diesel,turbo,four,sedan,rwd,front,107.9,...,152,idi,3.70,3.52,21.0,95.0,4150.0,28,33,16900.0
28,2,137.0,honda,gas,std,two,hatchback,fwd,front,86.6,...,92,1bbl,2.91,3.41,9.6,58.0,4800.0,49,54,6479.0
144,0,89.0,subaru,gas,std,four,wagon,fwd,front,97.0,...,108,2bbl,3.62,2.64,9.0,82.0,4800.0,28,32,7463.0


In [7]:
# build train input function
input_fn_train = tf.estimator.inputs.pandas_input_fn(
    x = pd.DataFrame({
        "make" : train_data["make"].values,
        "highway-mpg" : train_data["highway-mpg"].values,
        "curb-weight" : train_data["curb-weight"].values,
        "body-style" : train_data["body-style"].values
    }),
    y = pd.Series(train_data["price"].values),
    shuffle=False
)

In [10]:
feature_columns = [
    tf.feature_column.numeric_column("highway-mpg"),
    tf.feature_column.numeric_column("curb-weight"),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(key="body-style", vocabulary_list=[
            "hardtop", "wagon", "sedan", "hatchback", "convertible"
        ])
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_hash_bucket(key="make", hash_bucket_size=50),
        dimension=3
    )
]
estimator = tf.estimator.DNNRegressor(feature_columns=feature_columns, hidden_units=[20, 20])
estimator.train(input_fn=input_fn_train, steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1257eb400>, '_evaluation_master': '', '_log_step_count_steps': 100, '_save_summary_steps': 100, '_service': None, '_is_chief': True, '_model_dir': '/var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpht2cdr6s', '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_session_config': None, '_task_type': 'worker', '_keep_checkpoint_max': 5}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/55/qkrngbr949x1jzs0c0dt70wc000

In [11]:
input_fn_validation = tf.estimator.inputs.pandas_input_fn(
    x = pd.DataFrame({
        "make" : validation_data["make"].values,
        "highway-mpg" : validation_data["highway-mpg"].values,
        "curb-weight" : validation_data["curb-weight"].values,
        "body-style" : validation_data["body-style"].values
    }),
    y = pd.Series(validation_data["price"].values),
    shuffle=False
)

eval_result = estimator.evaluate(input_fn=input_fn_validation)

average_loss = eval_result["average_loss"]

# Convert MSE to Root Mean Square Error (RMSE).
print("\n" + 80 * "*")
print("\nRMS error for the test set: ${:.0f}"
    .format(average_loss**0.5))

print()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-20-07:00:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/55/qkrngbr949x1jzs0c0dt70wc0000gn/T/tmpht2cdr6s/model.ckpt-2
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-20-07:00:25
INFO:tensorflow:Saving dict for global step 2: average_loss = 127796890.0, global_step = 2, loss = 5239672300.0

********************************************************************************

RMS error for the test set: $11305

